In [1]:
from sys import path
path.append("..")
import networkx as nx
from ipywidgets import HBox, VBox, Label, HTML
import networkx as nx
from rdd.Node import Node
from rdd.RDD import *
import rdd.measures
from rdd.measures import *
from rdd.visualize import *
from rdd.other_sims import *
from rdd.ascos import *
from rdd.cos_sim import *

g1, g2 = nx.Graph(), nx.Graph()
g1.add_edges_from([
    (1, 2),
    (2, 3),
    (2, 4),
    (3, 4),
    (3, 6),
    (4, 5),
])
g2.add_edges_from([
    (1,2),
    (2,4),
    (2,3),
    (3,5),
    (4,5),
    (5,6)
])

G = nx.read_gml('../data/karate.gml', label='id')

real_network2 = nx.read_adjlist("../data/facebook_combined.txt", nodetype=int)
real_paths1 = nx.single_source_shortest_path(real_network2, 1, 2)
node_list1 = populate_node_list(real_paths1)
list_of_nodes = []
for node in node_list1:
    list_of_nodes.append(node.name)
subgraph = nodes_to_graph(real_network2, list_of_nodes)

pos = nx.spring_layout(G)
pos2 = nx.spring_layout(subgraph)

global_root_node = 1
global_radius = 5
global_markers = []
global_measure_list = [measures.triangles, 
                        measures.global_graph_degree,
                        measures.global_graph_clique,
                        measures.triangles,
                        measures.local_graph_degree,
                        measures.local_graph_clique]

In [2]:
ascos = visualize_ascos(G, global_root_node, pos)
ascos = add_markers(ascos, global_markers)
#ascos.show(config={'scrollZoom' : True})
df_ascos = get_ascos(G, global_root_node)

simrank = visualize_simrank(G, global_root_node, pos)
simrank = add_markers(simrank, global_markers)
#simrank.show(config={'scrollZoom' : True})
df_simrank = other_sims.simrank(G, global_root_node)

cosine_similarity = visualize_cosine_similarity(G, global_root_node, pos)
cosine_similarity = add_markers(cosine_similarity, global_markers)
#cosine_similarity.show(config={'scrollZoom' : True})
df_cosine_similarity = cos_sim.get_cosine(G, global_root_node)

vectoriezed = visualize_rdd_vector(G, global_root_node, global_radius, pos, global_measure_list)
vectoriezed = add_markers(vectoriezed, global_markers)
#vectoriezed.show(config={'scrollZoom' : True})
df_vectoriezed = measures.get_rdds_for_visuals_vector(G, global_root_node, global_measure_list, global_radius)

k_means_simrank = visualize_rdd_vector_kmeans_others(G, 1, ['simrank'], other_sims.simrank, pos)
k_means_simrank =  add_markers(k_means_simrank, global_markers)
k_means_simrank.show(config={'scrollZoom' : True})

k_means_cosine = visualize_rdd_vector_kmeans_others(G, 1, ['cos_sim'], get_cosine, pos)
k_means_cosine = add_markers(k_means_cosine, global_markers)
k_means_cosine.show(config={'scrollZoom' : True})

k_means_ascos = visualize_rdd_vector_kmeans_others(G, 1, ['ascos'], get_ascos, pos)
k_means_ascos = add_markers(k_means_ascos, global_markers)
k_means_ascos.show(config={'scrollZoom' : True})

k_means_rdd = visualize_rdd_vector_kmeans(G, 1, global_radius, global_measure_list, pos)
k_means_rdd = add_markers(k_means_rdd, global_markers)
k_means_rdd.show(config={'scrollZoom' : True})

mean_shift_cosine = visualize_rdd_vector_mean_shift_other(G, global_root_node, ['cos_sim'], get_cosine, pos)
mean_shift_cosine = add_markers(mean_shift_cosine, global_markers)
# mean_shift_cosine.show(config={'scrollZoom' : True})

mean_shift_simrank = visualize_rdd_vector_mean_shift_other(G, global_root_node, ['simrank'], other_sims.simrank, pos)
mean_shift_simrank = add_markers(mean_shift_simrank, global_markers)
# mean_shift_simrank.show(config={'scrollZoom' : True})

mean_shift_ascos = visualize_rdd_vector_mean_shift_other(G, global_root_node, ['ascos'], get_ascos, pos)
mean_shift_ascos = add_markers(mean_shift_ascos, global_markers)
# mean_shift_ascos.show(config={'scrollZoom' : True})

mean_shift_rdd = visualize_rdd_vector_mean_shift(G, global_root_node, global_radius, global_measure_list, pos)
mean_shift_rdd = add_markers(mean_shift_rdd, global_markers)
# mean_shift_rdd.show(config={'scrollZoom' : True})

In [3]:
VBox([HBox([VBox([HTML("<center>RDD Vectorized"), vectoriezed]), VBox([HTML("<center>CosSim"), cosine_similarity])]),
      HBox([VBox([HTML("<center>SimRank"), simrank]), VBox([HTML("<center>Ascos"), ascos])])])

    'data': …

In [4]:
VBox([HBox([VBox([HTML("<center>KMeans RDD Vectorized"), k_means_rdd]), VBox([HTML("<center>KMeans CosSim"), k_means_cosine])]),
      HBox([VBox([HTML("<center>KMeans SimRank"), k_means_simrank]), VBox([HTML("<center>KMeans Ascos"), k_means_ascos])])])

    '…

In [5]:
VBox([HBox([VBox([HTML("<center>MeanShift RDD Vectorized"), mean_shift_rdd]), VBox([HTML("<center>MeanShift CosSim"), mean_shift_cosine])]),
      HBox([VBox([HTML("<center>MeanShift SimRank"), mean_shift_simrank]), VBox([HTML("<center>MeanShift Ascos"), mean_shift_ascos])])])

  …

In [6]:
ascos = visualize_ascos(subgraph, global_root_node, pos2)
ascos = add_markers(ascos, global_markers)
#ascos.show(config={'scrollZoom' : True})
#df_ascos = get_ascos(subgraph, global_root_node)

simrank = visualize_simrank(subgraph, global_root_node, pos2)
simrank = add_markers(simrank, global_markers)
#simrank.show(config={'scrollZoom' : True})
#df_simrank = other_sims.simrank(subgraph, global_root_node)

cosine_similarity = visualize_cosine_similarity(subgraph, global_root_node, pos2)
cosine_similarity = add_markers(cosine_similarity, global_markers)
#cosine_similarity.show(config={'scrollZoom' : True})
#df_cosine_similarity = cos_sim.get_cosine(subgraph, global_root_node)

vectoriezed = visualize_rdd_vector(subgraph, global_root_node, global_radius, pos2, global_measure_list)
vectoriezed = add_markers(vectoriezed, global_markers)
#vectoriezed.show(config={'scrollZoom' : True})
#df_vectoriezed = measures.get_rdds_for_visuals_vector(subgraph, global_root_node, global_measure_list, global_radius)

KeyboardInterrupt: 

In [ ]:
VBox([HBox([VBox([HTML("<center>RDD Vectorized"), vectoriezed]), VBox([HTML("<center>CosSim"), cosine_similarity])]),
      HBox([VBox([HTML("<center>SimRank"), simrank]), VBox([HTML("<center>Ascos"), ascos])])])